In [0]:
# Cell 0: Build Gold Transfer Variables Library
# This notebook aggregates silver transfer variables by definition_hash
# to create a deduplicated library of unique variable definitions

import json
from pyspark.sql import functions as F
from pyspark.sql.types import *
from delta.tables import DeltaTable
from datetime import datetime

print("="*80)
print("BUILD GOLD TRANSFER VARIABLES LIBRARY")
print("="*80)

In [0]:
# Cell 1: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog = globals_dict['catalog']
silver_schema = globals_dict['silver_schema']
gold_schema = globals_dict['gold_schema']

source_table = f"{catalog}.{silver_schema}.md_transfer_variable_field_normalized"
target_table = f"{catalog}.{gold_schema}.md_transfer_variables_library"

print(f"Source: {source_table}")
print(f"Target: {target_table}")
print(f"Created by: {created_by_principal}")
print(f"Job: {databricks_job_name} (ID: {databricks_job_id}, Run: {databricks_run_id})")

In [0]:
# Cell 2: Aggregate Silver to Gold
print(f"{'='*80}")
print("Building Gold Transfer Variables Library...")
print(f"{'='*80}")

# Read from silver, filter for COMPLETED and MANUAL_REVIEW_REQUIRED records
# Both statuses indicate the record was processed (MANUAL_REVIEW_REQUIRED just needs review)
df_silver = spark.table(source_table).filter(
    (F.col("row_status").isin("COMPLETED", "MANUAL_REVIEW_REQUIRED")) &
    (F.col("definition_hash").isNotNull())
)

silver_count = df_silver.count()
completed_count = df_silver.filter(F.col("row_status") == "COMPLETED").count()
review_count = df_silver.filter(F.col("row_status") == "MANUAL_REVIEW_REQUIRED").count()

print(f"Silver records total: {silver_count}")
print(f"  - COMPLETED: {completed_count}")
print(f"  - MANUAL_REVIEW_REQUIRED: {review_count}")

if silver_count == 0:
    print("No records to process. Exiting.")
    dbutils.notebook.exit("No completed or review-required records in silver table")

# Group by definition_hash and aggregate
df_gold = df_silver.groupBy("definition_hash").agg(
    # Take first non-null value for each field (they should be identical for same hash)
    F.first("transfer_variable_name").alias("transfer_variable_name"),
    F.first("transfer_variable_label").alias("transfer_variable_label"),
    F.first("variable_description").alias("variable_description"),
    F.first("format").alias("format"),
    F.first("anticipated_max_length").alias("anticipated_max_length"),
    F.first("populate_for_all_records").alias("populate_for_all_records"),
    F.first("transfer_file_key").alias("transfer_file_key"),
    F.first("example_values").alias("example_values"),
    F.first("codelist_values").alias("codelist_values"),
    
    # Aggregate usage tracking
    F.collect_list(
        F.struct(
            F.col("trial_id"),
            F.col("data_stream_type"),
            F.col("data_provider_name"),
            F.col("transfer_variable_id"),
            F.col("parent_document_id"),
            F.col("row_status"),
            F.col("created_ts").alias("first_seen_ts"),
            F.col("last_updated_ts").alias("last_seen_ts")
        )
    ).alias("used_in_trials"),
    
    # Usage metrics
    F.count("*").alias("usage_count"),
    F.sum(F.when(F.col("row_status") == "COMPLETED", 1).otherwise(0)).alias("completed_count"),
    F.sum(F.when(F.col("row_status") == "MANUAL_REVIEW_REQUIRED", 1).otherwise(0)).alias("review_required_count"),
    F.min("created_ts").alias("first_seen_ts"),
    F.max("last_updated_ts").alias("last_seen_ts")
).withColumn(
    # Set needs_review flag if any source records need review
    "needs_review", 
    F.col("review_required_count") > 0
).withColumn("created_by_principal", F.lit(created_by_principal)) \
 .withColumn("created_ts", F.current_timestamp()) \
 .withColumn("last_updated_by_principal", F.lit(created_by_principal)) \
 .withColumn("last_updated_ts", F.current_timestamp()) \
 .withColumn("databricks_job_id", F.lit(databricks_job_id)) \
 .withColumn("databricks_run_id", F.lit(databricks_run_id)) \
 .withColumn("databricks_job_name", F.lit(databricks_job_name))

unique_definitions = df_gold.count()
print(f"Unique definitions (by hash): {unique_definitions}")
dedup_ratio = silver_count / unique_definitions if unique_definitions > 0 else 0
print(f"Deduplication ratio: {silver_count}/{unique_definitions} = {dedup_ratio:.2f}x")
print(f"{'='*80}")

In [0]:
# Cell 3: Create/Merge into Gold Table
if not spark.catalog.tableExists(target_table):
    print(f"Creating gold table: {target_table}")
    df_gold.write.format("delta").mode("overwrite").saveAsTable(target_table)
    print(f"✓ Created with {unique_definitions} definitions")
else:
    print(f"Merging into existing gold table: {target_table}")
    
    # MERGE to update existing definitions and insert new ones
    delta_table = DeltaTable.forName(spark, target_table)
    
    delta_table.alias("target").merge(
        df_gold.alias("source"),
        "target.definition_hash = source.definition_hash"
    ).whenMatchedUpdate(set={
        "used_in_trials": F.col("source.used_in_trials"),
        "usage_count": F.col("source.usage_count"),
        "completed_count": F.col("source.completed_count"),
        "review_required_count": F.col("source.review_required_count"),
        "needs_review": F.col("source.needs_review"),
        "last_seen_ts": F.col("source.last_seen_ts"),
        "last_updated_by_principal": F.col("source.last_updated_by_principal"),
        "last_updated_ts": F.col("source.last_updated_ts"),
        "databricks_job_id": F.col("source.databricks_job_id"),
        "databricks_run_id": F.col("source.databricks_run_id"),
        "databricks_job_name": F.col("source.databricks_job_name")
    }).whenNotMatchedInsertAll().execute()
    
    print(f"✓ Merge complete")

print(f"{'='*80}")
print("Gold table build complete!")
print(f"{'='*80}")

In [0]:
# Cell 4: Verification and Summary
print("\n=== GOLD TABLE SUMMARY ===")
df_result = spark.table(target_table)
total_defs = df_result.count()
needs_review_count = df_result.filter(F.col("needs_review") == True).count()
fully_validated_count = total_defs - needs_review_count

print(f"Total unique definitions: {total_defs}")
print(f"  - Fully validated (COMPLETED only): {fully_validated_count}")
print(f"  - Needs review (has MANUAL_REVIEW_REQUIRED): {needs_review_count}")

print(f"\nTop 10 most used definitions:")
df_result.select(
    "transfer_variable_name",
    "format",
    "usage_count",
    "needs_review",
    F.substring("definition_hash", 1, 12).alias("hash_preview")
).orderBy(F.desc("usage_count")).limit(10).display()